# Retrieves song features from the Spotify API using the Rolling Stones Top 100 artists list

In [76]:
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import numpy as np
from tqdm.notebook import tqdm

In [68]:
client_credentials_manager = SpotifyClientCredentials(client_id='c979f37a064e455e8778dbcd331c2d5d', 
                                                      client_secret='dc6fcb571e72436e8d13df8e6799b833')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False

In [69]:
df = pd.read_csv("data/rollingstones.csv")
#df = df.iloc[1:]

In [70]:
df

,artist
0,The Band
1,Elton John
2,Run-DMC
3,Patti Smith
4,Janis Joplin
...,...
95,Eric Clapton
96,Howlin’ Wolf
97,The Allman Brothers Band
98,Queen


In [71]:
#Gets artist ids
artist_ids = []

for artist in df["artist"]:
    search = sp.search(q=artist, type="artist")
    results = search['artists']['items']
    if len(results) > 0:
            sp_artist = results[0]['id']
            artist_ids.append(sp_artist)

In [72]:
#Gets top tracks in Spotify's format
top_tracks = []

for artist_id in artist_ids:
    track = sp.artist_top_tracks(artist_id, country='US')
    top_tracks.append(track)

retrying ...2secs


In [73]:
#Packages data into DataFrame
tracks = []

for track, artist in zip(top_tracks, df["artist"]):
    for i in range(len(track['tracks'])):
        track_name = track['tracks'][i]['name']
        track_id = track['tracks'][i]['id']
        song = {"artist" : artist,
                "track" : track_name,
                "id" : track_id}
        tracks.append(song)
        
df = pd.DataFrame(tracks)
df.to_csv("rs100_songs")

In [75]:
df["id"][1]

'4QMSq1uq0JgvyKhEDp8wkx'

In [78]:
features = []

i = 0
while i < len(df): 
    features.append(sp.audio_features(df["id"][i]))
    i += 1
    
features = [item for items in features for item in items] #flattens list
audio_features = pd.DataFrame(features)
audio_features.drop(['type', 'uri', 'track_href', 'analysis_url'], axis=1, inplace=True)

songs = audio_features.join(df["artist"])
songs = songs.join(df["track"])
songs.to_csv("rs_song_features")

retrying ...2secs
retrying ...6secs
retrying ...7secs
retrying ...5secs
retrying ...7secs
retrying ...6secs
retrying ...7secs
